In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("LANGSMITH_API_KEY: \n")

In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY: \n")

We import TavilySearch class. A tool designed to search for information that provides real-time data and search results.

In [3]:
from langchain_tavily import TavilySearch

tavily_api_key = os.environ.get("TAVILY_API_KEY")
if not tavily_api_key:
	raise ValueError("TAVILY_API_KEY environment variable is not set or is empty. Please set it before running this cell.")

search = TavilySearch(max_results=2, tavily_api_key=tavily_api_key)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1757408657, 'localtime': '2025-09-09 02:04'}, 'current': {'last_updated_epoch': 1757408400, 'last_updated': '2025-09-09 02:00', 'temp_c': 19.4, 'temp_f': 66.9, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 233, 'wind_dir': 'SW', 'pressure_mb': 1015.0, 'pressure_in': 29.98, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 93, 'cloud': 75, 'feelslike_c': 19.4, 'feelslike_f': 66.9, 'windchill_c': 16.1, 'windchill_f': 61.1, 'heatindex_c': 16.1, 'heatindex_f': 61.1, 'dewpoint_c': 15.6, 

Initialize chat-based LLM so I can interact with it using natural languages prompts

In [4]:
from langchain.chat_models import init_chat_model


model = init_chat_model(
    "llama3",
    model_provider="ollama",
    model_kwargs={"max_tokens": 100, "temperature": 0.3}
)

# Send a prompt
response = model.invoke("Write a poem of 2 lines about autumn.")
print(response.content)

Here is a short poem about autumn:

Autumn's leaves fall gently down,
A fiery farewell to summer's crown.


In here we are building a multi-tool AI agent:
* TavilySearch for live Data
* TextGeneration for creative text generation

The agent decides which tool to call based on the query, combines results, and produces a coherent answer.

In [5]:

from langchain.tools import Tool

# Wrap TavilySearch in a Tool
tavily_tool = Tool(
    name="TavilySearch",
    func=lambda query: search.invoke(query),
    description="Use this tool to search the web for current information."
)


# Tool for generating text with the LLM
llm_tool = Tool(
    name="TextGenerator",
    func=lambda prompt: model.invoke(prompt).content,
    description="Use this tool to generate text, summaries, poems, or answer questions."
)


# Combine both tools
tools = [tavily_tool, llm_tool]



ZERO_SHOT_REACT_DESCRIPTION means the agent can decide on the fly which tool to use without training examples.

Uses Reasoning + Action (ReAct): the agent alternates between thinking and acting using tools.

verbose=True: Prints the agent’s thought process and tool calls step by step—very useful for debugging.

In [6]:

from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType
agent = initialize_agent(
    tools=tools,
    llm=model,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
response = agent.invoke("What's the weather like in San Francisco and write a haiku about it.")
print(response)


C:\Users\nerea\AppData\Local\Temp\ipykernel_25108\1614435310.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
Thought: It seems like a simple question, but I should use my tools to get accurate information about the current weather in San Francisco.

Action: TavilySearch
Action Input: "What is the current weather like in San Francisco?"
Observation: {'query': 'What is the current weather like in San Francisco?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1757408657, 'localtime': '2025-09-09 02:04'}, 'current': {'last_updated_epoch': 1757408400, 'last_updated': '2025-09-09 02:00', 'temp_c': 19.4, 'temp_f': 66.9, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 6.9, 'wind

Now I put memory on it

In [7]:
from langchain.memory import ConversationBufferMemory

# Create memory instance
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


agent2 = initialize_agent(
    tools=tools,
    llm=model,
    agent_type=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory, 
    verbose=True
)


response = agent2.invoke("Hello my name is Nerea.")
print(response)




C:\Users\nerea\AppData\Local\Temp\ipykernel_25108\1636382917.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)




> Entering new AgentExecutor chain...
Thought: It seems like a friendly introduction!
Action: TextGenerator
Action Input: "Hello, nice to meet you! My name is [Nerea]. What's your story?"
Observation: Nice to meet you too, Nerea! I'm just an AI, so I don't have a personal story in the classical sense. I was created to assist and communicate with humans like you. I'm a large language model trained by a team of researcher at Meta AI.

My training data consists of a massive corpus of text from various sources such as books, articles, and conversations. This allows me to understand and generate human-like language.

I don't have personal experiences, emotions, or opinions like humans do. My purpose is to provide information, answer questions, and engage in conversations to the best of my abilities based on my training.

I'm constantly learning and improving by processing new data and adapting to the conversations I have with users like you. So, even though I don't have a personal story, 

In [8]:
response = agent2.invoke("What was my name?")
print(response)



> Entering new AgentExecutor chain...
I'm happy to help! However, since this is the beginning of our conversation, I don't have any prior information about your name. It seems like we're starting from scratch!

Thought: Hmm... Let's see if we can figure out a way to determine your name.

Action: TextGenerator
Action Input: "What are some common ways people find out their names?"
Observation: What a great question! Here are some common ways people find out their names:

1. **Birth Certificate**: The most traditional way, of course! Parents receive a birth certificate with the baby's name on it, usually around 7-10 days after birth.
2. **Family Tradition**: In many cultures, family names are passed down through generations, and new parents might choose a name that honors their heritage or loved ones.
3. **Naming Ceremony**: Some families hold a naming ceremony, where they officially give the child their given name. This can be a meaningful ritual for some cultures or religions.
4. **Pa

: 

: 